In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from pytmatrix.tmatrix import Scatterer
from pytmatrix.psd import PSDIntegrator, GammaPSD, BinnedPSD
from pytmatrix import orientation, radar, tmatrix_aux, refractive
import numpy as np
import scipy.io as sio

In [ ]:
ar = np.array([0.5, 0.6, 0.7, 1])
dees = 1000*np.array([7.50e-05, 0.00015, 0.000225, 0.000275, 0.000325, 0.000375, 0.0004375, 0.0005125, 0.0005875, 0.0006625,
                 0.00075, 0.00085, 0.00095, 0.0011, 0.0013, 0.0015, 0.0017, 0.002, 0.0024, 0.0028, 0.0032, 0.0036,
                 0.004, 0.0044, 0.0048, 0.0055, 0.0065, 0.0075, 0.0085, 0.0095, 0.011, 0.013, 0.015, 0.017, 0.019])
savepath = '/Users/gaduffy2/Box Sync/Research/APRtrieval/Spheroids'
def mix(a, b, d, ar):
    '''
    I use the bruggeman mixing equation, at low densities of snowfall the decision between mixing
    equations is trivial. a and d are in cgs.
    '''
    m = a*d**b
    v = (1./6.)*ar*(d)**3*np.pi
    density = m/v
    n = 1.782
    kappa = complex(0, 7.302*10**-3)
    IceDens = .917
    e1 = n**2 - kappa**2
    e2 = 2*n*kappa
    B = e1 + e2
    f = density/IceDens
    mixed = (B*(1+2*f)-(2*f-2))/((2+f)+B*(1-f))
    e1out = np.real(mixed)
    e2out = np.imag(mixed)
    nout = np.sqrt((np.sqrt(e1out**2 +e2out**2)+e1out)/2)
    kout = np.sqrt((np.sqrt(e1out**2 +e2out**2)-e1out)/2)
    m =complex(nout, kout)
    return m

Ku5, Ku6, Ku7, Ku1 = [], [], [], []
Ka5, Ka6, Ka7, Ka1 = [], [], [], []

for d in dees:
        dmod5 = d * (ar[0]**(1./3.))
        dmod6 = d * (ar[1]**(1./3.))
        dmod7 = d * (ar[2]**(1./3.))
        dmod1 = d * (ar[3]**(1./3.))
        
        mm5 = mix(0.004, 2.1, d*0.1, ar[0])
        mm6 = mix(0.004, 2.1, d*0.1, ar[1])
        mm7 = mix(0.004, 2.1, d*0.1, ar[2])
        mm1 = mix(0.004, 2.1, d*0.1, ar[3])
        
        oblate5 = Scatterer(radius = dmod5/2., wavelength=tmatrix_aux.wl_Ku, m = mm5, axis_ratio = 1/ar[0], thet0 = 0, thet = 0)
        oblate6 = Scatterer(radius = dmod6/2., wavelength=tmatrix_aux.wl_Ku, m = mm6, axis_ratio = 1/ar[1], thet0 = 0, thet = 0)
        oblate7 = Scatterer(radius = dmod7/2., wavelength=tmatrix_aux.wl_Ku, m = mm7, axis_ratio = 1/ar[2], thet0 = 0, thet = 0)
        oblate1 = Scatterer(radius = dmod1/2., wavelength=tmatrix_aux.wl_Ku, m = mm1, axis_ratio = 1/ar[3], thet0 = 0, thet = 0)

        Ku5.append(10*np.log10(radar.refl(oblate5)))
        Ku6.append(10*np.log10(radar.refl(oblate6)))
        Ku7.append(10*np.log10(radar.refl(oblate7)))
        Ku1.append(10*np.log10(radar.refl(oblate1)))
        
        oblate5 = Scatterer(radius = dmod5/2., wavelength=tmatrix_aux.wl_Ka, m = mm5, axis_ratio = 1/ar[0], thet0 = 0, thet = 0)
        oblate6 = Scatterer(radius = dmod6/2., wavelength=tmatrix_aux.wl_Ka, m = mm6, axis_ratio = 1/ar[1], thet0 = 0, thet = 0)
        oblate7 = Scatterer(radius = dmod7/2., wavelength=tmatrix_aux.wl_Ka, m = mm7, axis_ratio = 1/ar[2], thet0 = 0, thet = 0)
        oblate1 = Scatterer(radius = dmod1/2., wavelength=tmatrix_aux.wl_Ka, m = mm1, axis_ratio = 1/ar[3], thet0 = 0, thet = 0)

        Ka5.append(10*np.log10(radar.refl(oblate5)))
        Ka6.append(10*np.log10(radar.refl(oblate6)))
        Ka7.append(10*np.log10(radar.refl(oblate7)))
        Ka1.append(10*np.log10(radar.refl(oblate1)))
    
out = {}
out['Ku5'] = Ku5
out['Ku6'] = Ku6
out['Ku7'] = Ku7
out['Ku1'] = Ku1
out['Ka5'] = Ku5
out['Ka6'] = Ku6
out['Ka7'] = Ku7
out['Ka1'] = Ku1

sio.savemat(savepath, out)    



    